In [373]:
%matplotlib inline
import pandas as pd
import numpy as np
import xgboost as xgb
from scipy.optimize import fmin_cobyla
from mlxtend.regressor import StackingRegressor, stacking_regression
from xgboost import XGBRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.model_selection import KFold, RandomizedSearchCV, GridSearchCV, cross_val_predict
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.pipeline import make_pipeline, BaseEstimator, TransformerMixin, FeatureUnion
from sklearn.preprocessing import Imputer, KernelCenterer, StandardScaler, PolynomialFeatures

In [238]:
def rmse(y_true, y_pred): return np.sqrt(mean_squared_error(y_true, y_pred))

In [380]:
def load_data():
    train = pd.read_csv('../input/train.csv')
    test = pd.read_csv('../input/test.csv')
    
    y = np.log(train['SalePrice'].values)
    ids_submission = test['Id'].values
    
    combined = train.append(test, ignore_index=True).drop(['SalePrice'], axis=1)
    
    ordered_levels = {
        "Alley": ["Grvl", "Pave"],
        "BsmtCond": ["Po", "Fa", "TA", "Gd"],
        "BsmtExposure": ["No", "Mn", "Av", "Gd"],
        "BsmtFinType1": ["Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"],
        "BsmtFinType2": ["Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"],
        "BsmtQual": ["Fa", "TA", "Gd", "Ex"],
        "CentralAir": ["N", "Y"],
        "Electrical": ["FuseP", "FuseF", "FuseA", "Mix", "SBrkr"],
        "ExterCond": ["Po", "Fa", "TA", "Gd", "Ex"],
        "ExterQual": ["Fa", "TA", "Gd", "Ex"],
        "Fence": ["MnWw", "GdWo", "MnPrv", "GdPrv"],
        "FireplaceQu": ["Po", "Fa", "TA", "Gd", "Ex"],
        'Functional': ['Sev', 'Maj2', 'Maj1', 'Mod', 'Min2', 'Min1', 'Typ'],
        "GarageCond": ["Po", "Fa", "TA", "Gd", "Ex"],
        "GarageFinish": ["Unf", "RFn", "Fin"],
        "GarageQual": ["Po", "Fa", "TA", "Gd", "Ex"],
        "HeatingQC": ["Po", "Fa", "TA", "Gd", "Ex"],
        "KitchenQual": ["Fa", "TA", "Gd", "Ex"],
        "LotShape": ["IR3", "IR2", "IR1", "Reg"],
        "PavedDrive": ["N", "P", "Y"],
        "PoolQC": ["Fa", "Gd", "Ex"],
        "Street": ["Grvl", "Pave"],   
        "Utilities": ["NoSeWa", "AllPub"]
    }
    
    for c in combined.columns:
        if combined[c].dtype == 'object':
            if c in ordered_levels:
                combined[c] = combined[c].astype('category', categories = ordered_levels[c], ordered=True)
            else:
                combined[c] = combined[c].astype('category')
                    
    X = combined.iloc[:train.shape[0],:]
    X_submission = combined.iloc[train.shape[0]:,:]
    
    return y, X, X_submission, ids_submission   

y, X, X_submission, ids_submission = load_data()

In [489]:
class RegressionBlend(BaseEstimator):
    def __init__(self, regressors, scorer, n_folds=10):
        self.regressors = regressors
        self.n_folds = n_folds
        self.scorer = scorer
    
    def __blended(self, p, x):
        """blend model results using weights(p)"""
        result = None
        for i in range(len(p)):
            result = result + p[i] * x[i] if result is not None else p[i] * x[i]
        result /= sum(p)
        return result       

    def fit(self, X, y):        
        def constraint(p, *args):
            """constrain to positive weights"""
            return min(p) - .0    
        def error(p, x, y):
            """error function to optimize"""
            preds = self.__blended(p, x)
            err = self.scorer(y, preds)
            return err 
        preds = []
        for regressor in self.regressors:
            regressor.fit(X,y)
            preds.append(cross_val_predict(regressor, X, y, cv=KFold(self.n_folds)))
        # initial weights
        p0 = np.ones(len(self.regressors))
        p = fmin_cobyla(error, p0, args=(preds, y), cons=[constraint], rhoend=1e-5)
        self.weights = [x/sum(p) for x in p]
        blended_pred = self.__blended(p, preds)
        self.score = rmse(y, blended_pred)
        return self
    
    def predict(self, X):
        preds = []
        for regressor in self.regressors:
            preds.append(regressor.predict(X))
        return self.__blended(self.weights, preds)

In [490]:
class ProcessTreeData(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        _X = X.copy()
        
        keep_columns = [
            'Heating', 'ScreenPorch', 'PoolQC', 'CentralAir', 
            'HeatingQC', 'WoodDeckSF', 'PavedDrive', 'Exterior1st', 
            'PoolArea', 'TotalBsmtSF', 'BldgType', 'LotArea', 'YearBuilt', 
            'Neighborhood', 'MSZoning', 'SaleCondition', 'GrLivArea', 
            'OverallQual', 'OverallCond', 'BsmtUnfSF', 'BsmtExposure',
            'Fireplaces', 'GarageArea','Condition1','FireplaceQu']
        
        _X.drop(list(set(X.columns) - set(keep_columns)), axis=1, inplace=True)
        
        if 'FireplaceQu' in X.columns:
            _X["HasFireplace"] = 1 - X["FireplaceQu"].isnull() * 1
                
        for c in _X.columns:
            if _X[c].dtype.name == 'category':
                if _X[c].cat.ordered:
                    _X[c] = _X[c].cat.codes
                    
        return pd.get_dummies(_X)

In [491]:
class ProcessLinearData(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        _X = X.copy()

        keep_columns = [
            'Heating', 'ScreenPorch', 'PoolQC', 'CentralAir', 
            'HeatingQC', 'WoodDeckSF', 'PavedDrive', 'Exterior1st', 
            'PoolArea', 'TotalBsmtSF', 'BldgType', 'LotArea', 'YearBuilt', 
            'Neighborhood', 'MSZoning', 'SaleCondition', 'GrLivArea', 
            'OverallQual', 'OverallCond', 'BsmtUnfSF', 'BsmtExposure',
            'Fireplaces', 'GarageArea','Condition1','FireplaceQu']
            
        _X.drop(list(set(X.columns) - set(keep_columns)), axis=1, inplace=True)
        
        if 'FireplaceQu' in X.columns:
            _X["HasFireplace"] = 1 - X["FireplaceQu"].isnull() * 1

        for c in _X.columns:
            if _X[c].dtype.name == 'category':
                if _X[c].cat.ordered:
                    _X[c] = _X[c].cat.codes
                    
        # skewed columns (>0.75)
        for c in ['LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF2', '1stFlrSF', 
                  'GrLivArea', 'KitchenAbvGr', 'OpenPorchSF', 'PoolArea', 'MiscVal']:
            if c in _X.columns:
                _X[c] = np.log1p(X[c])
                                            
        return pd.get_dummies(_X)

In [492]:
class DropColumn(BaseEstimator, TransformerMixin):
    def __init__(self, columns=None, column=None):
        self.column = column
        self.columns = columns
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        X = X.copy()
        if self.columns is not None:
            X.drop(self.columns, axis=1, inplace=True)
        if self.column is not None:
            X.drop([self.column], axis=1, inplace=True)                                            
        return X

In [493]:
class KeepColumn(BaseEstimator, TransformerMixin):
    def __init__(self, columns=None, column=None):
        self.column = column
        self.columns = columns
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        X = X.copy()
        keep_columns = [] if self.columns is None else self.columns[:]
        if self.column is not None:
            keep_columns.append(self.column)
        return X.select(lambda x: x in keep_columns, axis=1)

In [494]:
model_xgb = make_pipeline(ProcessTreeData(),
                          Imputer(strategy='most_frequent'),
                          XGBRegressor(silent = True, 
                                       objective='reg:linear', 
                                       seed=1773,
                                       max_depth=5,
                                       nthread=8,
                                       learning_rate=0.05,
                                       n_estimators=500,
                                       min_child_weight=1,
                                       subsample=0.65,
                                       colsample_bytree=0.65
                                      ))

model_gbm = make_pipeline(ProcessTreeData(),
                          Imputer(strategy='most_frequent'),
                          GradientBoostingRegressor(random_state=1773,
                                                    learning_rate=0.1,
                                                    max_depth=4,
                                                    max_features=0.7,
                                                    min_samples_leaf=1,
                                                    n_estimators=250,
                                                    subsample=0.75))

model_et = make_pipeline(ProcessTreeData(),
                         Imputer(strategy='most_frequent'),
                         ExtraTreesRegressor(n_estimators=250,
                                             max_depth=14, 
                                             n_jobs=8,
                                             random_state=1773, 
                                             max_features=0.7))

model_en = make_pipeline(ProcessLinearData(),
                         Imputer(strategy='most_frequent'),
                         StandardScaler(),
                         PolynomialFeatures(interaction_only=True),
                         ElasticNet(l1_ratio=0.4, alpha=0.0009, max_iter=5000))

#metalearner = SVR(kernel='rbf')
#metalearner = LinearRegression()

model_stacked = RegressionBlend(regressors=[model_xgb, model_gbm, model_et, model_en], scorer=rmse, n_folds=)

# model_stacked = StackingRegressor(regressors=[model_xgb, model_et, model_en], 
#                                   meta_regressor=metalearner)

# model_stacked = StackingRegressor(regressors=[model_xgb, model_gbm, model_et, model_en], 
#                                   meta_regressor=metalearner)

In [483]:
params = {}

folds = KFold(2, random_state=1773)

grid = GridSearchCV(estimator=model_stacked, 
                    #n_jobs=4,
                    param_grid=params, 
                     #n_iter=60, 
                     #random_state=1773,
                     scoring=make_scorer(rmse, greater_is_better=False), 
                     cv=folds,
                     refit=True)

In [484]:
## new base -0.114874667872
# base -0.115007
## drop columns ##
# 24	FireplaceQu	-0.115124
# 21	Fireplaces	-0.115966
# 5	WoodDeckSF	-0.116229
# 6	PavedDrive	-0.116352
# 8	PoolArea	-0.116526
# 10	BldgType	-0.116734
# 1	ScreenPorch	-0.116817
# 2	PoolQC	-0.116853
# 4	HeatingQC	-0.116921
# 7	Exterior1st	-0.116956
# 0	Heating	-0.117091
# 23	Condition1	-0.117285
# 20	BsmtExposure	-0.117779
# 3	CentralAir	-0.117933
# 11	LotArea	-0.118374
# 14	MSZoning	-0.118597
# 13	Neighborhood	-0.118616
# 22	GarageArea	-0.118815
# 15	SaleCondition	-0.119013
# 19	BsmtUnfSF	-0.120547
# 17	OverallQual	-0.120855
# 12	YearBuilt	-0.120858
# 9	TotalBsmtSF	-0.122659
# 18	OverallCond	-0.125446
# 16	GrLivArea	-0.141893

In [485]:
#grid.get_params()

In [486]:
#grid.get_params().keys()

In [487]:
#grid.estimator.get_params().keys()

In [495]:
%%time
model_stacked.fit(X,y)

#                     #n_jobs=4,
#                     param_grid=params, 
#                      #n_iter=60, 
#                      #random_state=1773,
#                      scoring=make_scorer(rmse, greater_is_better=False), 
#                      cv=folds,
#                      refit=True)

CPU times: user 2min 10s, sys: 2.53 s, total: 2min 12s
Wall time: 1min 14s


RegressionBlend(n_folds=10,
        regressors=[Pipeline(steps=[('processtreedata', ProcessTreeData()), ('imputer', Imputer(axis=0, copy=True, missing_values='NaN', strategy='most_frequent',
    verbose=0)), ('xgbregressor', XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.65,
       gamma=0, learning_rate=0.05, ma...se, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False))])],
        scorer=<function rmse at 0x11231d8c0>)

In [498]:
model_stacked.weights

[0.47423642905842228,
 0.31768891292669066,
 0.12204718348595336,
 0.086027474528933751]

In [488]:
%%time
grid.fit(X,y)
print 'best score:', grid.best_score_
# print 'best parameters:', grid.best_params_

best score: -0.122429658387
CPU times: user 5min 47s, sys: 12.2 s, total: 5min 59s
Wall time: 3min 15s


In [370]:
grid.best_estimator_.coef_

array([ 0.21529208,  0.8514377 , -0.05355732])

In [361]:
# pd.DataFrame({'params': [x['dropcolumn__column'] for x in grid.cv_results_['params']], 
#               'test_score': grid.cv_results_['mean_test_score']})\
#   .sort_values('test_score', ascending=False)

In [339]:
preds_sub = grid.predict(X_submission)

In [340]:
pd.DataFrame({'Id': ids_submission, 'SalePrice': np.exp(preds_sub)}).to_csv('../ensemble/models/stacked_sub_2.csv', index=False)

In [350]:
def get_oof_preds(model, X, y, X_sub, n_folds=10, n_iter=1, seed=1234):
    from random import Random
    from scipy.stats import hmean
    preds = np.zeros((np.shape(X)[0], n_iter))
    preds_sub = np.zeros((np.shape(X_sub)[0], n_iter))
    rng = Random(seed)
    for i in range(n_iter):
        rs = rng.randint(1,9999)
        folds = KFold(n_folds, shuffle=True, random_state=rs)
        preds_sub_j = np.zeros((np.shape(X_sub)[0], n_folds))
        #print 'iter: {}'.format(i)
        for j, (train_index, test_index) in enumerate(folds.split(X)):
            if type(X) == pd.DataFrame:
                X_train = X.iloc[train_index, :]
                X_test = X.iloc[test_index, :]
            else:
                X_train = X[train_index, :]
                X_test = X[test_index, :]                
            y_train = y[train_index]
            model.fit(X_train, y_train)
            preds[test_index, i] = model.predict(X_test)
            preds_sub_j[:,j] = model.predict(X_sub)
        preds_sub[:, i] = hmean(np.clip(preds_sub_j, 1e-5, 14), axis=1)
    return hmean(preds, axis=1), hmean(preds_sub, axis=1)

In [352]:
sub_preds = get_oof_preds(model_stacked, X, y, X_submission)

In [353]:
pd.DataFrame({'Id': ids_submission, 'SalePrice': np.exp(sub_preds[1])}).to_csv('../ensemble/models/stacked_sub_3.csv', index=False)